In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
data.head(3)

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0


In [ ]:
data_original = data.copy

In [ ]:
clean_data = data.drop(['StudentID', 'GPA'], axis=1)
clean_data.head(3)

,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GradeClass
0,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.0
1,18,0,0,1,15.408756,0,0,1,0,0,0,0,1.0
2,15,0,2,3,4.210570,26,0,2,0,0,0,0,4.0


In [ ]:
'''seperating the data into features and targets'''
features = clean_data.drop('GradeClass', axis=1)
targets = clean_data['GradeClass']

In [ ]:
'''scaling the data points'''
from sklearn.preprocessing import StandardScaler

std_sclr = StandardScaler()
scaled_features = std_sclr.fit_transform(features)

In [ ]:
'''reverting the data into a dataframe'''
new_features = pd.DataFrame(scaled_features, columns=features.columns)

In [ ]:
'''over sampling the data points'''
from imblearn.over_sampling import RandomOverSampler

over_sampler = RandomOverSampler(random_state=0)
X,y = over_sampler.fit_resample(new_features, targets)

In [ ]:
'''spliting the data in training data and test data'''
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.33, random_state=0, shuffle=False)

In [ ]:
'''cross validation'''
from sklearn.model_selection import StratifiedKFold
sKFold = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

In [ ]:
'''model'''
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


models = {
    'Logistic_regresion': LogisticRegression(),
    'Random_forest': RandomForestClassifier(),
    'naive_bayes': GaussianNB(),
    'support_Vector_machine': SVC()
}

In [ ]:
'''measuring tools'''
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import accuracy_score as accur_score

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_curve, classification_report

# Initialize lists for storing scores
accuracy_scores = []
precision_scores = []
recall_scores = []

for X_index, X_cv_index in sKFold.split(X_train, y_train):
    for name, model in models.items():

        train_X = X_train.iloc[X_index]
        train_y = y_train.iloc[X_index]
        train_cv_X = X_train.iloc[X_cv_index]
        train_cv_y = y_train.iloc[X_cv_index]

        model.fit(train_X, train_y)

        y_pred = model.predict(train_cv_X)

        # Use classification_report for multiclass problems
        report = classification_report(train_cv_y, y_pred, output_dict=True)
        accuracy = accuracy_score(train_cv_y, y_pred)

        accuracy_scores.append(accuracy)
        # Extract precision, recall for each class from the report
        precision_scores.append(report['weighted avg']['precision'])
        recall_scores.append(report['weighted avg']['recall'])

        print(f'Fold accuracy for {name}: {accuracy}')
        print(f'Classification report for {name}:\n {report}')

# Calculate average accuracy
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(f'\nAverage Accuracy: {average_accuracy}')

Fold accuracy for Logistic_regresion: 0.5665024630541872
Classification report for Logistic_regresion:
 {'0.0': {'precision': 0.4775, 'recall': 0.7860082304526749, 'f1-score': 0.5940902021772939, 'support': 243}, '1.0': {'precision': 0.43010752688172044, 'recall': 0.24242424242424243, 'f1-score': 0.31007751937984496, 'support': 165}, '2.0': {'precision': 0.18181818181818182, 'recall': 0.02531645569620253, 'f1-score': 0.044444444444444446, 'support': 79}, '3.0': {'precision': 0.2, 'recall': 0.012195121951219513, 'f1-score': 0.02298850574712644, 'support': 82}, '4.0': {'precision': 0.7458745874587459, 'recall': 0.9300411522633745, 'f1-score': 0.8278388278388279, 'support': 243}, 'accuracy': 0.5665024630541872, 'macro avg': {'precision': 0.4070600592317296, 'recall': 0.39919704055754274, 'f1-score': 0.3598878999175075, 'support': 812}, 'weighted avg': {'precision': 0.4913933535118171, 'recall': 0.5665024630541872, 'f1-score': 0.4951819132684329, 'support': 812}}
Fold accuracy for Random_f